In [1]:
#############################跑很多遍都是一个结果
#######################
import json
import pandas as pd
import pickle as pkl
import re, os, glob
import numpy as np
import re
import sys
from collections import defaultdict
import nltk
import string
from gensim.models import Phrases
from gensim.utils import SaveLoad
from gensim.models.phrases import Phraser
from nltk.corpus import stopwords  # Import the stop word list
import timeit
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet as wn
from nltk.stem import PorterStemmer, WordNetLemmatizer
# from nltk import pos_tag, word_tokenize
from nltk.tag import PerceptronTagger
from collections import OrderedDict

In [2]:
# Pywsd's Lemmatizer.
porter = PorterStemmer()
wnl = WordNetLemmatizer()
tagger = PerceptronTagger()
pos_tag = tagger.tag
tokenizer = RegexpTokenizer(r'\.|\w+')


def lemmatize(ambiguous_word, pos=None, neverstem=True,
              lemmatizer=wnl, stemmer=porter):
    """
    Tries to convert a surface word into lemma, and if lemmatize word is not in
    wordnet then try and convert surface word into its stem.
    This is to handle the case where users input a surface word as an ambiguous
    word and the surface word is a not a lemma.
    """
    if pos:
        lemma = lemmatizer.lemmatize(ambiguous_word, pos=pos)
    else:
        lemma = lemmatizer.lemmatize(ambiguous_word)
    stem = stemmer.stem(ambiguous_word)
    # Ensure that ambiguous word is a lemma.
    if not wn.synsets(lemma):
        if neverstem:
            return ambiguous_word
        if not wn.synsets(stem):
            return ambiguous_word
        else:
            return stem
    else:
        return lemma


def penn2morphy(penntag, returnNone=False):
    morphy_tag = {'NN': wn.NOUN, 'JJ': wn.ADJ,
                  'VB': wn.VERB, 'RB': wn.ADV}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return None if returnNone else ''


def word_tokenize(text, tokenize=tokenizer):
    return tokenize.tokenize(text.lower())  # doesn't remove stopwords
    # return [w for w in tokenize.tokenize(text.lower()) if not w in stopwords.words("english")]


def lemmatize_sentence(sentence, neverstem=False, keepWordPOS=False,
                       tokenizer=word_tokenize, postagger=pos_tag,
                       lemmatizer=wnl, stemmer=porter):
    words, lemmas, poss = [], [], []
    for word, pos in postagger(sentence):  # change tokenizer(sentence) to sentence
        pos = penn2morphy(pos)
        lemmas.append(lemmatize(word.lower(), pos, neverstem,
                                lemmatizer, stemmer))
        poss.append(pos)
        words.append(word)
    if keepWordPOS:
        return words, lemmas, [None if i == '' else i for i in poss]
    return lemmas

In [3]:
def fil_sub(reviews, fill=None):
    words = word_tokenize(str(reviews))  # tokenize and remove punctuation
    if any(x in words for x in fill):
        return reviews
    return np.nan


def filter_news(title, bigram, trigram):
    fill = set()
    # sp = pd.read_csv('sp500.csv', names=['Symbol', 'Name', 'market', 'number'])
    sp = pd.read_csv('sp100.csv')
    sp = sp.dropna(axis=0, how='any')  # drop the row that's incomplete
    sp['Symbol'] = sp['Symbol'].str.lower()
    ##TODO the following steps make sure that all the symbols and company names are included,despite format
    # TODO Do the phrase
    sp['phrase'] = sp['Name'].apply(convert, args=(bigram, trigram, False, True, False)).replace([' \. ', ' \.'], ' ',
                                                                                                 regex=True)
    sp['phrase'] = sp['phrase'].replace(['\\bu s\\b', '\\bs p\\b', '\\bat t\\b', '\\b\d+\\b'],
                                        ['u_s', 's_p', 'at_t', 'xxx'],
                                        regex=True)  # becareful of using inplace
    sp['phrase'] = sp['phrase'].replace(
        ['class_\w', 'class \w', ' inc\\b', ' corp\\b', ' company\\b', ' com\\b', ' group\\b', ' store\\b',
         ' co\\b',
         ' plc\\b', "\\b\w\\b"],
        '',
        regex=True)
    # TODO not do the phrase
    sp['no_phrase'] = sp['Name'].apply(convert, args=(bigram, trigram, False, False, False)).replace([' \. ', ' \.'],
                                                                                                     ' ',
                                                                                                     regex=True)
    sp['no_phrase'] = sp['no_phrase'].replace(['\\bu s\\b', '\\bs p\\b', '\\bat t\\b', '\\b\d+\\b'],
                                              ['u_s', 's_p', 'at_t', 'xxx'],
                                              regex=True)  # becareful of using inplace
    sp['no_phrase'] = sp['no_phrase'].replace(
        ['class_\w', 'class \w', ' inc\\b', ' corp\\b', ' company\\b', ' com\\b', ' group\\b', ' store\\b',
         ' co\\b',
         ' plc\\b', "\\b\w\\b"],
        '',
        regex=True)
    ##TODO connet all together
    sp['connect'] = sp['Name'].apply(convert, args=(bigram, trigram, False, False, False)).replace(
        ['\\bu s\\b', '\\bs p\\b', '\\bat t\\b', '\\b\d+\\b'],
        ['u_s', 's_p', 'at_t', 'xxx'],
        regex=True).replace(
        [' \. ', ' \.', "\\b\w\\b", 'class'], ' ',
        regex=True).apply(
        lambda x: '_'.join(x.split()))
    ##TODO remove last word
    sp['rm_last'] = sp['Name'].apply(convert, args=(bigram, trigram, False, False, False)).replace(
        ['\\bu s\\b', '\\bs p\\b', '\\bat t\\b', '\\b\d+\\b'],
        ['u_s', 's_p', 'at_t', 'xxx'],
        regex=True).replace(
        [' \. ', ' \.', "\\b\w\\b"], ' ',
        regex=True).apply(lambda x: ' '.join(x.split()[:-1]))
    sp['rm_last_connect'] = sp['rm_last'].apply(lambda x: '_'.join(x.split()))
    sp['phrase_connect'] = sp['phrase'].apply(lambda x: '_'.join(x.split()))
    sp['no_phrase_connect'] = sp['no_phrase'].apply(lambda x: '_'.join(x.split()))
    fill.update(sp['no_phrase'].values.tolist())
    fill.update(sp['phrase'].values.tolist())
    fill.update(sp['Symbol'].values.tolist())
    fill.update(sp['Name'].str.lower().values.tolist())
    fill.update(sp['connect'].values.tolist())
    fill.update(sp['rm_last'].values.tolist())
    fill.update(sp['rm_last_connect'].values.tolist())
    fill.update(sp['phrase_connect'].values.tolist())
    fill.update(sp['no_phrase_connect'].values.tolist())
    fill.update(
        ['us', 'standardpoor', 'america', 'exxon', 'wall_st', 'wall_street', 'wall st', 'wall street', 'u_s', 'j_j',
         'h_r', 'at_t', 'p_g', 'r_d', 'dow_jones', 'b_l', 'h_m', 's_p', 'amazon', 'exxon_mobil'])
    fill = list(filter(None, fill))
    print('length of total filter', len(fill))
    title_select = title.groupby(title.index).apply(list).apply(pd.Series).fillna('')
    for i in title_select.index:
        if len(list(filter(None, title_select.loc[i].values))) > 1:
            try:
                title.loc[i] = title.loc[i].apply(fil_sub, fill=fill)
            except AttributeError as e:
                print(i + ' ' + str(e))

    return title

In [4]:
types = ['title', 'abstract','article']
start = timeit.default_timer()
print('start processing Ding_abstract data')
article = defaultdict(list)
with open('news_title/reuters_news.json',"r") as data:  # title+ abstract + article
    title = pd.DataFrame(json.loads(line, strict=False) for line in data).set_index('date')
    
with open('news_title/bloomberg_news.json',"r") as data:  # title+ abstract + article
    bloomberg = pd.DataFrame(json.loads(line, strict=False) for line in data).set_index('date')

title = title.append(bloomberg)
print('original shape', title.shape) #original shape (553870, 3)

start processing Ding_abstract data
original shape (553870, 3)


In [5]:
title[:10]

,title,abstract,article
date,,,
2011-11-16,Olympus plans $3.4 billion debt reduction: Nik...,(Reuters) - Japan's disgraced Olympus Corp ( ...,Olympus's bank creditors are crucial to its p...
2011-11-16,Swiss cabinet in move to inform U.S. bank clie...,ZURICH (Reuters) - The Swiss cabinet approve...,"""The amendment should ensure that the procedu..."
2011-11-16,Euro zone Oct inflation 3 percent; seen falling.,BRUSSELS (Reuters) - Euro zone inflation hel...,"Now at a three-year high, consumer prices in ..."
2011-11-16,"Qantas hopeful on deal with unions, shares rally.",SYDNEY (Reuters) - Australia's Qantas Airway...,"The optimistic comments, just days before the..."
2011-11-16,More euro zone integration a must for euro sur...,"STRASBOURG, France (Reuters) - The euro zone...","""Without this increased integration, converge..."
2011-11-16,for traders.,LONDON (Reuters) - Life is not easy for the ...,There are no pumped-up traders cheering from ...
2011-11-16,"Citigroup could cut up to 3,000 jobs: report.",NEW YORK (Reuters) - Citigroup In ( C.N )c i...,The job cuts are intended to help cut Citigro...
2011-11-16,Economy shows signs of momentum in 4th quarter.,OTTAWA (Reuters) - Canadian consumers spent ...,"Retail sales grew by 1 percent in the month, ..."
2011-11-16,Seaway oil pipe to be reversed after Enbridge ...,(Reuters) - Enbridge Inc ( ENB.TO ) surged pa...,Enbridge and Enterprise Products Partners ( E...


In [6]:
##TODO don't do this if want to use character level
#title['title'] = title['title'].apply(lambda x: [x])
#title['abstract'] = title['abstract'].apply(lambda x: [x])
##TODO fill empty with nan,only use when using abstract and article
#title = title.applymap(lambda x: np.nan if not x else x)
##TODO drop nan
title = title.dropna(axis=0, how='any')  # drop the row that's incomplete
title = title.sort_index()
print('after dropping empty abstract and articles, first drop nan', title.shape)

after dropping empty abstract and articles, first drop nan (553870, 3)


In [7]:
title = title.replace(['corrected', 'UPDATE\s\d-', "'s", 'update xxx', 'wrapup xxx', 'factbox', 'instant view', 'snap analysis','exclusive','timeline', 'highlights', 'correction', 'scenarios', 'analysis'], '', regex=True)  # becareful of using inplace
#title = title.replace(['corrected', 'UPDATE\s\d-', "'s", 'update xxx', 'wrapup xxx', 'factbox', 'instant view', 'snap analysis','exclusive','timeline', 'highlights', 'correction', 'scenarios', 'analysis'], '', regex=True)  

In [8]:
title = title.replace(['\\bus\\b', '\\bu s\\b', "\\bu_s\\b"], 'america', regex=True)  # becareful of using inplace
#title = title.replace(['us', 'u s', "u_s"], 'america', regex=True)  # becareful of using inplace

In [9]:
title = title.replace(['\\buk\\b', '\\bu k\\b', "\\bu_k\\b"], 'british', regex=True)  # becareful of using inplace
#title = title.replace(['uk', 'u k', "u_k"], 'british', regex=True)  # becareful of using inplace

In [10]:
title = title.replace(['\\bs p\\b', '\\bs_p\\b', "\\bs_p xxx\\b"], 'standardpoor',regex=True)  # becareful of using inplace
#title = title.replace(['s p', 's_p', "s_p xxx"], 'standardpoor',regex=True)  # becareful of using inplace

In [11]:
title = title.replace(['\ss\s'], ' ', regex=True)  # becareful of using inplace
#title = title.replace(['\ss\s'], ' ', regex=True)  # becareful of using inplace

In [12]:
title = title.replace(
    ['\\bwo not\\b', '\\bbln\\b', '\\bmln\\b', '\\bpct\\b', '\\bs korea\\b', '\\bn korea\\b', '3rd', '2nd', '1st',
     '4th','\\bq1\\b','\\bq2\\b', '\\bq3\\b', '\\bq4\\b', '\\bwall street\\b', '\\bisnt\\b', '\\b1st_', '\\b2nd_', '\\b3rd_',
     '\\b4th_','\\bfirst_', '\\bsecond_', '\\bthird_', '\\bfourth_'],
    ['would not', 'billion', 'million', 'percent', 'south_korea', 'north_korea', 'third', 'second', 'first',
     'fourth','first quarter', 'second quarter', 'third quarter', 'fourth quarter', 'wall_street', 'is not', 'first ',
     'second ', 'third ', 'fourth ', 'first ', 'second ', 'third ', 'fourth '],regex=True)  # becareful of using inplace
#title = title.replace(
 #   ['wo not', 'bln', 'mln', 'pct', 's korea', 'n korea', '3rd', '2nd', '1st',
 #    '4th','q1','q2', 'q3', 'q4', 'wall street', 'isnt', '1st_', '2nd_', '3rd_',
 #    '4th_','first_', 'second_', 'third_', 'fourth_'],
 #   ['would not', 'billion', 'million', 'percent', 'south_korea', 'north_korea', 'third', 'second', 'first',
 #    'fourth','first quarter', 'second quarter', 'third quarter', 'fourth quarter', 'wall_street', 'is not', 'first ',
 #    'second ', 'third ', 'fourth ', 'first ', 'second ', 'third ', 'fourth '],
 #   regex=True)  # becareful of using inplace

In [13]:
title = title.replace(['qtr', 'quarterly'], 'quarter', regex=True)  # becareful of using inplace

In [14]:
title = title.replace(['\\bj j\\b', '\\bh r\\b', '\\bat t\\b', '\\bp g\\b', '\\br d\\b', 'dow jones', '\\bdow\\b', '\\bb l\\b','\\bh m\\b', ' _ ', '\\b\st\\b'],
    ['j_j', 'h_r', 'at_t', 'p_g', 'r_d', 'dow_jones', 'dow_jones', 'b_l', 'h_m', ' and ', 'street'],regex=True)  # becareful of using inplace
#title = title.replace(
 #   ['j j', 'h r', 'at t', 'p g', 'r d', 'dow jones', 'dow', 'b l','h m', ' _ ', '\st'],
  #  ['j_j', 'h_r', 'at_t', 'p_g', 'r_d', 'dow_jones', 'dow_jones', 'b_l', 'h_m', ' and ', 'street'],regex=True)  # becareful of using inplace

In [15]:
title = title.replace(['\d+\S\d+', 'xxx xxx', 'xxx xxx xxx'], 'xxx',regex=True)  # becareful of using inplace

In [16]:
#title --->list?
title

,title,abstract,article
date,,,
2006-10-20,"Hey buddy, can you spare $xxx for a Google share?",SAN FRANCISCO/NEW YORK (Reuters) - Wall Stre...,Brokers were responding to quarter results o...
2006-10-20,Exxon Mobil offers plan to end Alaska dispute.,"ANCHORAGE, Alaska (Reuters) - Exxon Mobil ( ...",In a proposal sent earlier this week to the ...
2006-10-20,"Inco Net Soars on Higher Metal Prices, Breakup...","Inco Ltd., the Canadian nickel producer being ...","Net income jumped to $xxx million, or $xxx a s..."
2006-10-21,AOL CEO says sales may shrink for two years -p...,FRANKFURT (Reuters) - Internet service provi...,"""Maybe another two years, you are right ther..."
2006-10-21,"Jim Cramer: Diageo, Anheuser-Busch, Monster Wo...",Jim Cramer recommended that viewers buy share...,Diageo is similar to Guess? Inc. (GES) as a ...
...,...,...,...
2013-11-26,"DBS, ABN Said to Make Final Bids for SocGen’s ...",DBS Group Holdings Ltd. (DBS) and ABN Amro ...,Bids submitted yesterday have valued the unit ...
2013-11-26,Taiwan Bond Yield Falls to Three-Week Low as O...,"Taiwan ’s 10-year bonds gained, driving the yi...",Industrial production climbed xxx percent in O...
2013-11-26,Billionaire Said to Consider IPO for Owner of ...,Malaysian billionaire Vincent Tan is consideri...,Tan is holding preliminary discussions with ba...


In [17]:
#title.reset_index().to_csv("news_set/title.csv", index=False, encoding='utf-8')
title['title'].type()  #numpy,Series

AttributeError: 'Series' object has no attribute 'type'

In [18]:
def convert(reviews, bigram, trigram, remove_stopwords=True, phrase=True, lemma=True):
    # letters_only = re.sub("[^a-zA-Z0-9]", " ", str(reviews))
    # words = list(filter(None, letters_only.lower().split()))
    words = word_tokenize(reviews)  # tokenize and remove punctuation
    if remove_stopwords:  # remove stopwords
        words = [w for w in words if not w in stopwords.words("english")]
    if phrase is True:
        words = trigram[bigram[words]]  # to phrase
    if lemma is True:
        words = lemmatize_sentence(words)  # lemma
    return " ".join(words)  # + ' .'

In [19]:
def build_dictionary(filepaths, dst_path, vocab):
    #build_dictionary(title, path, 'title'/'abstract')
    word_freqs = OrderedDict()
    print('processing the vocab cases for ' + vocab)
    for i in filepaths[vocab]:  #delete value
        words_in = i.strip().split(' ')
        for w in words_in:
            if w not in word_freqs:
                word_freqs[w] = 0
            word_freqs[w] += 1
    print('before dropping the words that occurs less than 5 times', len(word_freqs))
    
    for key, value in list(word_freqs.items()):
        if value < 5:
            del word_freqs[key]
    
    words = list(word_freqs.keys())
    freqs = list(word_freqs.values())

    sorted_idx = np.argsort(freqs)
    sorted_words = [words[ii] for ii in sorted_idx[::-1]]

    worddict = OrderedDict()
    worddict['_PAD_'] = 0  # default, padding
    worddict['_UNK_'] = 1  # out-of-vocabulary
    worddict['_BOS_'] = 2  # begin of sentence token
    worddict['_EOS_'] = 3  # end of sentence token

    for ii, ww in enumerate(sorted_words):
        worddict[ww] = ii + 4

    with open('vocab_cased_' + vocab + 'd.pickle', 'wb') as f:
        pkl.dump(worddict, f)

    print('Dict size', len(worddict))
    print('Done')

In [20]:
bigram = SaveLoad.load("big_phrase.pickle")
trigram = SaveLoad.load("trig_phrase.pickle")

In [21]:
##TODO only work on title
"""
print('finish dealing with manual replace')
title['title'] = title['title'].apply(convert, args=(bigram, trigram, False, False, True))
title = title.drop_duplicates(subset='title', keep='first')
print('after dropping ' + 'title' + ' is {}'.format(title.shape))
"""

for i in types:
    # title[i] = title[i].str.replace('[{}]'.format(string.punctuation), ' ')
    title[i] = title[i].apply(convert, args=(bigram, trigram, False, False, False))
    title = title.drop_duplicates(subset=[i], keep='first')
    print('after dropping ' + i + ' is {}'.format(title.shape)) #(529922, 3),(511271, 3)

after dropping title is (529922, 3)


/userhome/35/yqdeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


after dropping abstract is (511271, 3)
after dropping article is (474311, 3)


In [22]:
##TODO change the news that's too short to nan, only use when processing abstract and article
title['title'] = title['title'].apply(lambda x: x if len(x.split()) > 2 else np.nan)
title['abstract'] = title['abstract'].apply(lambda x: x if len(x) >= 100 else np.nan)
title['article'] = title['article'].apply(lambda x: x if len(x) >= 100 else np.nan)
##TODO filter the news
print('start the filter process')
title['title'] = filter_news(title['title'], bigram, trigram)
##TODO drop nan
title = title.dropna(axis=0, how='any')  # drop the row that's incomplete
print('after dropping incomplete rows, second drop nan', title.shape)
#after dropping incomplete rows, second drop nan (59857, 3)
#title = title.applymap(lambda x: x.split('.'))

start the filter process
length of total filter 476
after dropping incomplete rows, second drop nan (58075, 3)


In [23]:
title  #numpy

,title,abstract,article
date,,,
2006-10-20,exxon mobil offers plan to end alaska dispute .,anchorage alaska reuters exxon mobil xom . n o...,in a proposal sent earlier this week to the al...
2006-10-23,royal dutch bid for shell canada low sharehold...,calgary alberta reuters one of shell canada lt...,garey aitken a portfolio manager at bissett in...
2006-10-23,dow ends at record high texas instruments off ...,new york reuters u . s . stocks rallied on mon...,shares of wal mart hit an 18 month high after ...
2006-10-23,shell looks to buy out canada unit for c xxx b...,calgary london reuters royal dutch shell plc r...,shell said in a statement on monday it had adv...
2006-10-23,ford is reviewing all products brands ceo .,detroit reuters ford motor co . is committed t...,the automaker will look at the entire product ...
...,...,...,...
2013-11-25,apple eaton google wockhardt intellectual prop...,apple inc . aapl won more than xxx million fro...,after a weeklong trial in san jose california ...
2013-11-26,rubber drops to two week low after forecast su...,rubber declined to a two week low after a fore...,futures for delivery in april on the tokyo com...
2013-11-26,japan credit risk drops to more than two month...,the cost of insuring corporate bonds from non ...,the markit itraxx japan index fell xxx of a ba...


In [25]:
##TODO the news start from 2006-10-20 to 2013-11-20 reuters, bloomberg to 2013-11-26
#train = title['2006-10-20':'2012-06-21']#['2006-10-20':'2012-07-21']  # from the beginning 2006-10-20 plus 30 days after, just in case
#validate = title['2012-05-22':'2013-04-11']  # plus 30 days before and after, just in case
#test = title['2013-02-12':'2013-11-20']  # plus 30 days before and after, just in case
stop = timeit.default_timer()
print("run time for ding:", stop - start)
# os.chdir('/home/jialong/Documents/phrase_embedding/yunke_' + types + '/')
#train.reset_index().to_csv("news_set/train.csv", index=False, encoding='utf-8')
#validate.reset_index().to_csv("news_set/validate.csv", index=False, encoding='utf-8')
#test.reset_index().to_csv("news_set/test.csv", index=False, encoding='utf-8')
path = 'news_set/'
for i in types:
    build_dictionary(title, path, i)

run time for ding: 2825.3510674647987
processing the vocab cases for title
before dropping the words that occurs less than 5 times 24718
Dict size 7584
Done
processing the vocab cases for abstract
before dropping the words that occurs less than 5 times 40897
Dict size 14184
Done
processing the vocab cases for article
before dropping the words that occurs less than 5 times 128528
Dict size 55707
Done


In [26]:
#build_dictionary()
title

,title,abstract,article
date,,,
2006-10-20,exxon mobil offers plan to end alaska dispute .,anchorage alaska reuters exxon mobil xom . n o...,in a proposal sent earlier this week to the al...
2006-10-23,royal dutch bid for shell canada low sharehold...,calgary alberta reuters one of shell canada lt...,garey aitken a portfolio manager at bissett in...
2006-10-23,dow ends at record high texas instruments off ...,new york reuters u . s . stocks rallied on mon...,shares of wal mart hit an 18 month high after ...
2006-10-23,shell looks to buy out canada unit for c xxx b...,calgary london reuters royal dutch shell plc r...,shell said in a statement on monday it had adv...
2006-10-23,ford is reviewing all products brands ceo .,detroit reuters ford motor co . is committed t...,the automaker will look at the entire product ...
...,...,...,...
2013-11-25,apple eaton google wockhardt intellectual prop...,apple inc . aapl won more than xxx million fro...,after a weeklong trial in san jose california ...
2013-11-26,rubber drops to two week low after forecast su...,rubber declined to a two week low after a fore...,futures for delivery in april on the tokyo com...
2013-11-26,japan credit risk drops to more than two month...,the cost of insuring corporate bonds from non ...,the markit itraxx japan index fell xxx of a ba...


In [ ]:
l1 = []
>>> for elem in l:
        l1.extend(elem.strip().split(';'))  
>>> l1